In [4]:
!pip install selenium

   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------- ----------------------------- 2.4/9.4 MB 15.0 MB/s eta 0:00:01
   ------------------------ --------------- 5.8/9.4 MB 14.7 MB/s eta 0:00:01
   ------------------------------------ --- 8.7/9.4 MB 14.9 MB/s eta 0:00:01
   ---------------------------------------- 9.4/9.4 MB 14.5 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.7.0
    Uninstalling websocket-client-1.7.0:
      Successfully uninstalled websocket-client-1.7.0


In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

driver_path = 'C:/Users/Landon Hawley/Downloads/chromedriver-win64/chromedriver.exe'
service = Service(driver_path)

driver = webdriver.Chrome(service=service)

# Open the URL
url = "https://www.espn.com/nfl/stats/player/_/season/2019/seasontype/2"
driver.get(url)

time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

# Get the page source after JavaScript has loaded
html_content = driver.page_source

driver.quit()

In [7]:
soup = BeautifulSoup(html_content, "lxml")

namerank_columns = ['Rank', 'Name/Team']
namerank_df = pd.DataFrame(columns = namerank_columns)

namerank_div = soup.find(name = 'div', attrs = {'class' : 'ResponsiveTable ResponsiveTable--fixed-left mt4 Table2__title--remove-capitalization'})
table = namerank_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    namerank_series = pd.Series(data, index = namerank_columns)
    namerank_df = pd.concat([namerank_df, namerank_series.to_frame().T], ignore_index=True)

In [8]:
display(namerank_df)

,Rank,Name/Team
0,1,Jameis WinstonTB
1,2,Dak PrescottDAL
2,3,Jared GoffLAR
3,4,Philip RiversLAC
4,5,Matt RyanATL
...,...,...
101,91,Kareem HuntCLE
102,91,Christian McCaffreyCAR
103,91,Preston WilliamsMIA
104,91,Steven SimsWSH


In [9]:
import re


team_abbrs = [
    "ARI", "ATL", "BAL", "BUF", "CAR", "CHI", "CIN", "CLE", "DAL", "DEN",
    "DET", "GB", "HOU", "IND", "JAX", "KC", "LAC", "LAR", "LV", "MIA",
    "MIN", "NE", "NO", "NYG", "NYJ", "PHI", "PIT", "SEA", "SF", "TB",
    "TEN", "WSH", "OAK"
]


team_pattern = r'(' + '|'.join(team_abbrs) + r')$'


def split_string_first(s):
    match = re.search(team_pattern, s)
    if match:
        return s[:match.start()]
    return s


def split_string_second(s):
    match = re.search(team_pattern, s)
    if match:
        return match.group()
    return ''


namerank_df['Name'] = namerank_df['Name/Team'].apply(split_string_first)
namerank_df['Team'] = namerank_df['Name/Team'].apply(split_string_second)
namerank_df.drop('Name/Team', inplace=True, axis=1)

In [10]:
namerank_df

,Rank,Name,Team
0,1,Jameis Winston,TB
1,2,Dak Prescott,DAL
2,3,Jared Goff,LAR
3,4,Philip Rivers,LAC
4,5,Matt Ryan,ATL
...,...,...,...
101,91,Kareem Hunt,CLE
102,91,Christian McCaffrey,CAR
103,91,Preston Williams,MIA
104,91,Steven Sims,WSH


In [11]:
stats_columns = ['Position', 'Games Played', 'Completions', 'Passing Attempts', 'Completion Percentage', 'Passing Yards',
                 'Avg Yards per Pass Attempt', 'Passing Yards per Game', 'Longest Pass', 'Passing Touchdowns', 'Interceptions',
                 'Total Sacks', 'Sack Yards Lost', 'Adjusted QBR', 'Passer Rating']
stats_df = pd.DataFrame(columns = stats_columns)

stats_div = soup.find(name = 'div', attrs = {'class' : 'Table__Scroller'})
table = stats_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    stats_series = pd.Series(data, index = stats_columns)
    stats_df = pd.concat([stats_df, stats_series.to_frame().T], ignore_index=True)

stats_df

,Position,Games Played,Completions,Passing Attempts,Completion Percentage,Passing Yards,Avg Yards per Pass Attempt,Passing Yards per Game,Longest Pass,Passing Touchdowns,Interceptions,Total Sacks,Sack Yards Lost,Adjusted QBR,Passer Rating
0,QB,16,380,626,60.7,"5,109",8.2,319.3,71,33,30,47,282,55.7,60.4
1,QB,16,388,596,65.1,"4,902",8.2,306.4,62,30,11,23,151,71.9,60.4
2,QB,16,394,626,62.9,"4,638",7.4,289.9,66,22,16,22,170,50.6,56.3
3,QB,16,390,591,66.0,"4,615",7.8,288.4,84,23,20,34,222,50.5,56.3
4,QB,15,408,616,66.2,"4,466",7.3,297.7,93,26,14,48,316,59.6,56.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,RB,8,0,0,0.0,0,0.0,0.0,0,0,0,1,8,0.0,39.6
102,RB,16,0,2,0.0,0,0.0,0.0,0,0,0,0,0,0.2,39.6
103,WR,8,0,0,0.0,0,0.0,0.0,0,0,0,1,1,0.7,39.6
104,WR,16,0,1,0.0,0,0.0,0.0,0,0,0,0,0,,39.6


In [12]:
total_stats = pd.concat([namerank_df, stats_df], axis=1)
total_stats

,Rank,Name,Team,Position,Games Played,Completions,Passing Attempts,Completion Percentage,Passing Yards,Avg Yards per Pass Attempt,Passing Yards per Game,Longest Pass,Passing Touchdowns,Interceptions,Total Sacks,Sack Yards Lost,Adjusted QBR,Passer Rating
0,1,Jameis Winston,TB,QB,16,380,626,60.7,"5,109",8.2,319.3,71,33,30,47,282,55.7,60.4
1,2,Dak Prescott,DAL,QB,16,388,596,65.1,"4,902",8.2,306.4,62,30,11,23,151,71.9,60.4
2,3,Jared Goff,LAR,QB,16,394,626,62.9,"4,638",7.4,289.9,66,22,16,22,170,50.6,56.3
3,4,Philip Rivers,LAC,QB,16,390,591,66.0,"4,615",7.8,288.4,84,23,20,34,222,50.5,56.3
4,5,Matt Ryan,ATL,QB,15,408,616,66.2,"4,466",7.3,297.7,93,26,14,48,316,59.6,56.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,91,Kareem Hunt,CLE,RB,8,0,0,0.0,0,0.0,0.0,0,0,0,1,8,0.0,39.6
102,91,Christian McCaffrey,CAR,RB,16,0,2,0.0,0,0.0,0.0,0,0,0,0,0,0.2,39.6
103,91,Preston Williams,MIA,WR,8,0,0,0.0,0,0.0,0.0,0,0,0,1,1,0.7,39.6
104,91,Steven Sims,WSH,WR,16,0,1,0.0,0,0.0,0.0,0,0,0,0,0,,39.6


In [14]:
total_stats.drop(total_stats[total_stats['Name'].str.contains('/')].index, inplace = True)
total_stats

,Rank,Name,Team,Position,Games Played,Completions,Passing Attempts,Completion Percentage,Passing Yards,Avg Yards per Pass Attempt,Passing Yards per Game,Longest Pass,Passing Touchdowns,Interceptions,Total Sacks,Sack Yards Lost,Adjusted QBR,Passer Rating
0,1,Jameis Winston,TB,QB,16,380,626,60.7,"5,109",8.2,319.3,71,33,30,47,282,55.7,60.4
1,2,Dak Prescott,DAL,QB,16,388,596,65.1,"4,902",8.2,306.4,62,30,11,23,151,71.9,60.4
2,3,Jared Goff,LAR,QB,16,394,626,62.9,"4,638",7.4,289.9,66,22,16,22,170,50.6,56.3
3,4,Philip Rivers,LAC,QB,16,390,591,66.0,"4,615",7.8,288.4,84,23,20,34,222,50.5,56.3
4,5,Matt Ryan,ATL,QB,15,408,616,66.2,"4,466",7.3,297.7,93,26,14,48,316,59.6,56.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,91,Kareem Hunt,CLE,RB,8,0,0,0.0,0,0.0,0.0,0,0,0,1,8,0.0,39.6
102,91,Christian McCaffrey,CAR,RB,16,0,2,0.0,0,0.0,0.0,0,0,0,0,0,0.2,39.6
103,91,Preston Williams,MIA,WR,8,0,0,0.0,0,0.0,0.0,0,0,0,1,1,0.7,39.6
104,91,Steven Sims,WSH,WR,16,0,1,0.0,0,0.0,0.0,0,0,0,0,0,,39.6


In [18]:
total_stats.to_csv("Passing_Stats_2019.csv", index = False)
print("Data saved to Passing_Stats_2019.csv")

Data saved to Passing_Stats_2019.csv
